In [1]:
# Check device number
!ls -ltrh /dev/video*

crw-rw----+ 1 root video 81, 1 Aug 28 16:18 /dev/video1
crw-rw----+ 1 root video 81, 2 Aug 28 16:18 /dev/video2


In [2]:
from ultralytics import YOLO
import torch
import os

print(torch.__version__, "cuda:", torch.cuda.is_available())

#make ft16 tensorrt file
model = YOLO("yolo.pt", task="pose")
model.export(format="engine", int8=True, data="../cali_dataset/cali_data.yaml")

os.rename("yolo.onnx", "yolo_int8_onnx")
os.rename("yolo.engine", "yolo_int8.engine")


2.8.0 cuda: True
WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics 8.3.187 🚀 Python-3.10.12 torch-2.8.0 CUDA:0 (Orin, 7620MiB)
YOLO11n-pose summary (fused): 109 layers, 2,956,000 parameters, 0 gradients, 7.8 GFLOPs

PyTorch: starting from 'yolo.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 68, 8400) (6.0 MB)
requirements: Ultralytics requirement ['onnxruntime-gpu'] not found, attempting AutoUpdate...


ERROR: Could not find a version that satisfies the requirement onnxruntime-gpu (from versions: none)


WARNING ⚠️ Retry 1/2 failed: Command 'pip install --no-cache-dir "onnxruntime-gpu" ' returned non-zero exit status 1.


ERROR: No matching distribution found for onnxruntime-gpu
ERROR: Could not find a version that satisfies the requirement onnxruntime-gpu (from versions: none)


WARNING ⚠️ Retry 2/2 failed: Command 'pip install --no-cache-dir "onnxruntime-gpu" ' returned non-zero exit status 1.
WARNING ⚠️ requirements: ❌ Command 'pip install --no-cache-dir "onnxruntime-gpu" ' returned non-zero exit status 1.


ERROR: No matching distribution found for onnxruntime-gpu



ONNX: starting export with onnx 1.12.0 opset 19...
ONNX: slimming with onnxslim 0.1.59...
WARNING ⚠️ ONNX: simplifier failure: module 'onnx.helper' has no attribute 'get_all_tensor_dtypes'
ONNX: export success ✅ 8.2s, saved as 'yolo.onnx' (11.6 MB)

TensorRT: starting export with TensorRT 10.3.0...
TensorRT: collecting INT8 calibration images from 'data=../cali_dataset/cali_data.yaml'
Fast image access ✅ (ping: 0.0±0.0 ms, read: 234.2±65.6 MB/s, size: 12.7 KB)
Scanning /home/wonmin/project/hand_keypoints/cali_dataset/labels.cache... 50 images, 0 backgrounds, 2 corrupt: 100% ━━━━━━━━━━━━ 50/50 28934.2it/s 0.0s0s
/home/wonmin/project/hand_keypoints/cali_dataset/images/IMG_00019284.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0245]
/home/wonmin/project/hand_keypoints/cali_dataset/images/IMG_00034950.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0261]
WARNING ⚠️ TensorRT: >300 images recommended for INT8 ca

In [1]:
from flask import Flask, Response
from ultralytics import YOLO
import cv2

# YOLO 모델 불러오기 (best.pt or .engine 가능)
model = YOLO("yolo_int8.engine", task="pose")

c_index = 1 #Camera index can be changed

cap = cv2.VideoCapture(c_index) 
assert cap.isOpened(), "Camera open failed."

app = Flask(__name__)

def gen_frames():
    while True:
        ret, frame = cap.read() 
        if not ret:
            break

        # YOLO inference
        results = model.predict(source=frame, imgsz=640,verbose=False)
        annotated = results[0].plot()  # draw inference results

        # JPEG encoding
        ret, buffer = cv2.imencode('.jpg', annotated)
        if not ret:
            continue #If encoding is failed, skip

        # MJPEG streaming: MJPEG = Motion JPEG
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + buffer.tobytes() + b'\r\n')

@app.route('/')
def video_feed():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == "__main__":
    try:
        app.run(host="0.0.0.0", port=8000, debug=False, use_reloader=False)
    finally:
        cap.release()
        cv2.destroyAllWindows()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.30.1.50:8000
Press CTRL+C to quit


Loading yolo_int8.engine for TensorRT inference...
[08/28/2025-16:24:53] [TRT] [I] Loaded engine size: 5 MiB
[08/28/2025-16:24:53] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +12, now: CPU 0, GPU 15 (MiB)


172.30.1.93 - - [28/Aug/2025 16:24:55] "GET / HTTP/1.1" 200 -
